In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import pickle
# Load the dataset
data_path = 'Grenades.csv'
grenades_df = pd.read_csv(data_path)

# Define categorical and numerical features
categorical_features = ['Type', 'Material', 'Explosive Type', 'Activation Mechanism']
numerical_features = ['Weight (kg)', 'Blast Radius (m)', 'Effectiveness (%)', 'Price (INR)']

# Handle missing values
imputer = SimpleImputer(strategy='most_frequent')
grenades_df[categorical_features] = imputer.fit_transform(grenades_df[categorical_features])
grenades_df[numerical_features] = imputer.fit_transform(grenades_df[numerical_features])

# Encode categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_categorical = encoder.fit_transform(grenades_df[categorical_features])

# Scale numerical features
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(grenades_df[numerical_features])

# Combine processed features
processed_features = np.hstack((encoded_categorical, scaled_numerical))

# Define the recommendation function
def recommend_grenades(user_input):
    """Recommend top 5 grenades based on user input."""
    # Process user input
    user_categorical = [user_input[feature] for feature in categorical_features]
    user_numerical = [user_input[feature] for feature in numerical_features]
    
    # Encode and scale user input
    encoded_user_categorical = encoder.transform([user_categorical])
    scaled_user_numerical = scaler.transform([user_numerical])
    
    user_processed = np.hstack((encoded_user_categorical, scaled_user_numerical))
    
    # Calculate similarity scores (cosine similarity)
    similarity_scores = np.dot(processed_features, user_processed.T).flatten() / (
        np.linalg.norm(processed_features, axis=1) * np.linalg.norm(user_processed)
    )
    
    # Get top 5 recommendations
    top_indices = np.argsort(similarity_scores)[-5:][::-1]
    recommendations = grenades_df.iloc[top_indices]
    
    return recommendations

# Example user input
user_input_example = {
    'Type': 'Fragmentation',
    'Material': 'Steel',
    'Weight (kg)': 0.6,
    'Explosive Type': 'RDX',
    'Blast Radius (m)': 20,
    'Activation Mechanism': 'Impact',
    'Effectiveness (%)': 85,
    'Price (INR)': 1500
}

# Get recommendations
recommended_grenades = recommend_grenades(user_input_example)

# Display the recommendations with sequential ranking
print("Top 5 Recommended Grenades:\n")
for rank, (_, row) in enumerate(recommended_grenades.iterrows(), start=1):
    print(f"Grenade {rank}:")
    for column in grenades_df.columns:
        print(f"  {column}: {row[column]}")
    print()


Top 5 Recommended Grenades:

Grenade 1:
  Grenade ID: 3
  Name: Hand Grenade
  Type: Fragmentation
  Material: Steel
  Weight (kg): 0.55
  Explosive Type: RDX
  Blast Radius (m): 20.0
  Safety Mechanism: Pull Pin
  Activation Mechanism: Impact
  Usage: Combat
  Country of Origin: India
  Introduced Year: 1980
  Effectiveness (%): 95.0
  Price (INR): 1500.0
  Operational Environment: Urban/Combat
  Durability (years): 5
  Popularity: High
  Legal Restrictions: Restricted

Grenade 2:
  Grenade ID: 16
  Name: Cluster Grenade
  Type: Fragmentation
  Material: Steel
  Weight (kg): 0.7
  Explosive Type: TNT
  Blast Radius (m): 25.0
  Safety Mechanism: Pull Pin
  Activation Mechanism: Impact
  Usage: Combat
  Country of Origin: India
  Introduced Year: 2000
  Effectiveness (%): 85.0
  Price (INR): 1800.0
  Operational Environment: Urban/Combat
  Durability (years): 6
  Popularity: Medium
  Legal Restrictions: Restricted

Grenade 3:
  Grenade ID: 2
  Name: FM-2
  Type: Fragmentation
  Material

C:\Users\Survesh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
C:\Users\Survesh\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [2]:
# Create a dictionary with all the necessary components of the model
grenade_model = {
    'grenades_df': grenades_df,
    'encoder': encoder,
    'scaler': scaler,
    'processed_features': processed_features,
    'categorical_features': categorical_features,
    'numerical_features': numerical_features
}

# Serialize (pickle) the model to a file
with open('grenade_recommendation_model.pkl', 'wb') as file:
    pickle.dump(grenade_model, file)

print("Grenade recommendation model has been pickled successfully!")

Grenade recommendation model has been pickled successfully!
